## 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace' # Path to our workspace
SCRIPTS_PATH = 'Tensorflow/scripts' # Where generate tf record script is
APIMODEL_PATH = 'Tensorflow/models' # tf objection detection library is downloaded here
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations' # label records and tf records
IMAGE_PATH = WORKSPACE_PATH + '/images' # labelled images
MODEL_PATH = WORKSPACE_PATH + '/models' # where trained model will be present
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models' # Where our downloaded pre-trained model is saved
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config' # tf object detection library has a config file containing information about Deep Learning Model to be used
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/' # All check-points are saved

## 1. Create Label Map

In [2]:
# Categories in which images are labelled are set here
labels = [{'name':'Mask','id':1},{'name':'NoMask','id':2}]

In [3]:
labels

[{'name': 'Mask', 'id': 1}, {'name': 'NoMask', 'id': 2}]

In [4]:
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## 2. Create TF Records

In [9]:
#!pip install tensorflow-object-detection-api

     ------------------------------------- 577.4/577.4 kB 70.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 237.5/237.5 kB 114.5 kB/s eta 0:00:00
     -------------------------------------- 54.4/54.4 kB 113.0 kB/s eta 0:00:00
     -------------------------------------- 51.1/51.1 kB 104.4 kB/s eta 0:00:00
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-py3-none-any.whl size=844512 sha256=dd426d277b4cad6a9da72f9c4e54034f05271131b4a471436bdcae8ada5f2bbb
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\38\6f\73\3c0c70f9a8219c31c96c495b686af1416154b60707fd96a4cd
Successfully built tensorflow-object-detection-api


Since I have Tensorflow==2.0, I made these changes to the following files:
1. ***object_detection\utils\label_map_util.py*** `tf.gfile` is replaced by `tf.io.gfile` (line 132)
2. ***generate_tfrecord.py*** in (line 61 & 62)

    `label_map = label_map_util.load_labelmap(args.labels_path)`
    
    `label_map_dict = label_map_util.get_label_map_dict(label_map)`
    
    is replaced by a single statement
    
    `label_map_dict = label_map_util.get_label_map_dict(args.labels_path)`

In [9]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record


In [10]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


## 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [13]:
# I cloned this repository using Git Application
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.
